In [ ]:
import random, time
import numpy as np
import pandas as pd
import multiprocessing as mp
import numba as nb

# Topics

1. Python Functions
2. NumPy
3. Multiprocessing
4. Cython
5. Numba

## Python Functions


```
def function_name(optional_parameters):
    function_code
    return optional_return_variables
```

Functions encapsulate a set of instructions that can be reused. The following tips for faster Python all benefit from writing code that is well functionalized. Further, using functions are generally a good coding practice as they allow for the blocks of code to be written once and then reused many times. Then if the code needs to be updated it only needs to be updated once and not many times.

### Example: Functions estimating π via a Monte-Carlo algorithm

In [ ]:
def monte_carlo_pi(points):
    s = 0
    for _ in range(points):
        x = random.random()**2
        y = random.random()**2
        if x + y < 1:
            s += 1
    return 4. * float(s) / float(points)

def sample_points(p):
    s = []
    for i in range(p):
        s.append(monte_carlo_pi(10**p))
    return s

def print_sample(s):
    for p in s:
        print(p)

In [ ]:
print_sample(sample_points(7))

### Task: Create functions estimating π via a Gauss–Legendre algorithm

Initial:

$a_0 = 1$

$b_0 = \frac{1}{\sqrt{2}}$

$t_0 = \frac{1}{4}$

$p_0 = 1$

Loop until $a_{n}$ and $b_{n}$ difference meets threashold:

$a_{n+1} = \frac{a_{n}+b_{n}}{2}$

$b_{n+1} = \sqrt{a_{n}b_{n}}$

$t_{n+1} = t_{n}-p_{n}\sqrt{a_{n}-a_{n+1}}$

$p_{n+1} = 2p_{n}$

$\pi \approx \frac{(a_{n+1}+b_{n+1})^2}{4t_{n+1}}$

## multiprocessing

`multiprocessing` is a package that supports spawning processes using an API similar to the threading module. The multiprocessing package offers both local and remote concurrency, effectively side-stepping the Global Interpreter Lock by using subprocesses instead of threads. Due to this, the multiprocessing module allows the programmer to fully leverage multiple processors on a given machine. It runs on both Unix and Windows.

The multiprocessing module also introduces APIs which do not have analogs in the threading module. A prime example of this is the `Pool` object which offers a convenient means of parallelizing the execution of a function across multiple input values, distributing the input data across processes (data parallelism). The following example demonstrates the common practice of defining such functions in a module so that child processes can successfully import that module. This basic example of data parallelism using `Pool`:

In [ ]:
# Import multiprocessing module
import multiprocessing as mp

# Define function that can be run in parallel
def printHello(thread):
    print("Hello from process " + str(thread) + "!")

def run_tasks():
    jobs = []
    # Launch processes
    for i in range(4):
        p = mp.Process(target = printHello, args = (i,))
        jobs.append(p)
        p.start()

In [ ]:
run_tasks()

In [ ]:
def point(points):
    np.random.seed()
    hit = 0
    for i in range(0, points):
        x = np.power(np.random.rand(1), 2)
        y = np.power(np.random.rand(1), 2)
        if np.sqrt(x + y) <= 1:
            hit += 1
    return hit

def run_parallel_pi(points=100, cores=1):
    # Setup variables
    points_per_core = int(points / cores)
    n = [points_per_core] * cores
    n[0] = points_per_core + (points - (points_per_core * cores))

    # Launch processes
    pool = mp.Pool(processes = cores)
    results = pool.map(point, n)

    # Display results
    print("Hits: {0}".format(results))
    print("Pi Est: {0}".format(4. * np.sum(results) / float(points)))

In [ ]:
run_parallel_pi()

### Task: Create a parallel AXPY implementation

A times X plus Y, where A is a scalar and X and Y are vectors.

## [NumPy](https://numpy.org)

NumPy is the fundamental package for scientific computing with Python. It contains among other things:

* a powerful N-dimensional array object
* sophisticated (broadcasting) functions
* tools for integrating C/C++ and Fortran code
* useful linear algebra, Fourier transform, and random number capabilities

Besides its obvious scientific uses, NumPy can also be used as an efficient multi-dimensional container of generic data. Arbitrary data-types can be defined. This allows NumPy to seamlessly and speedily integrate with a wide variety of databases.

Many NumPy functions are compiled libraries using fast BLAS and LAPACK implementations. These functions and functions built on top of NumPy functions will likely run faster than pure Python implementations.

### Example: Python Matrix-Matrix Multiplication

In [ ]:
def initialize_matrix(m, n, fill=0):
    matrix = []
    for i in range(m):
        row = []
        for j in range(n):
            if fill == 0:
                f = 0
            else:
                f = random.random()
            row.append(f)
        matrix.append(row)
    return matrix

def gemm(matrix_a, matrix_b):
    m = len(matrix_a[:][0])
    n = len(matrix_b[:][0])
    p = len(matrix_a[0][:])
    matrix_c = initialize_matrix(m, p)
    for i in range(m):
        for j in range(p):
            for k in range(n):
                matrix_c[i][j] += matrix_a[i][k]*matrix_b[k][j]
    return matrix_c

In [ ]:
matrix_a = [[1,2],[3,4]]
matrix_b = [[1,2],[3,4]]
matrix_c = gemm(matrix_a, matrix_b)
print(matrix_c)

In [ ]:
matrix_a = np.array(matrix_a)
matrix_b = np.array(matrix_b)
matrix_c = np.matmul(matrix_a, matrix_b)
print(matrix_c)

### Task: Create a function that compares the speed of Python and NumPy implementations of matrix-matrix multiplication

Use random square matrices of size 500, 1000, and 1500. Time the intialization and multiplication collectively using `time.time()`. Hints: [`rand()`](https://docs.scipy.org/doc/numpy-1.14.0/reference/generated/numpy.random.rand.html) and [`matmul()`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.matmul.html).

## Cython

Cython is an optimising static compiler for both the Python programming language and the extended Cython programming language (based on Pyrex). It makes writing C extensions for Python as easy as Python itself.

Cython gives you the combined power of Python and C to let you

* Write Python code that calls back and forth from and to C or C++ code natively at any point.
* Easily tune readable Python code into plain C performance by adding static type declarations, also in Python syntax.
* Use combined source code level debugging to find bugs in your Python, Cython and C code.
* Interact efficiently with large data sets, e.g. using multi-dimensional NumPy arrays.
* Quickly build your applications within the large, mature and widely used CPython ecosystem.
* Integrate natively with existing code and data from legacy, low-level or high-performance libraries and applications.


In [ ]:
def primes(nb_primes):
    n = 2
    p = []
    while len(p) < nb_primes:
        for i in p:
            if n % i == 0:
                break
        else:
            p.append(n)
        n += 1
    return p

In [ ]:
%load_ext Cython

In [ ]:
%%cython
def c_primes(int nb_primes):
    cdef int n, i, len_p
    cdef int p[1000]
    if nb_primes > 1000:
        nb_primes = 1000
    len_p = 0
    n = 2
    while len_p < nb_primes:
        for i in p[:len_p]:
            if n % i == 0:
                break
        else:
            p[len_p] = n
            len_p += 1
        n += 1
    result_as_list  = [prime for prime in p[:len_p]]
    return result_as_list

In [ ]:
%timeit primes(100)

In [ ]:
%timeit c_primes(100)

### Task: Create a function that compares the speed of Python and Cython implementations of Fibonacci sequence

$F_0 = 0$, $F_1 = 1$, and $F_n = F_{n-1}+F_{n-2}$

## Numba

Numba translates Python functions to optimized machine code at runtime using the industry-standard LLVM compiler library. Numba-compiled numerical algorithms in Python can approach the speeds of C or FORTRAN.

### Example: Lennard-Jones

In [ ]:
import numpy as np

def lj_numpy(r):
    sr6 = (1./r)**6
    pot = 4.*(sr6*sr6 - sr6)
    return pot

def distances_numpy(cluster):
    diff = cluster[:, np.newaxis, :] - cluster[np.newaxis, :, :]
    mat = np.sqrt((diff * diff).sum(-1))
    return mat

def potential_numpy(cluster):
    d = distances_numpy(cluster)
    dtri = np.triu(d)
    energy = lj_numpy(dtri[dtri > 1e-6]).sum()
    return energy

def make_cluster(natoms, radius=20, seed=1981):
    np.random.seed(seed)
    cluster = np.random.normal(0, radius, size=(natoms, 3)) - 0.5
    return cluster

In [ ]:
cluster = make_cluster(10000)
%timeit potential_numpy(cluster)

In [ ]:
import numpy as np
import numba

@numba.njit
def lj_numba_array(r):
    sr6 = (1./r)**6
    pot = 4.*(sr6*sr6 - sr6)
    return pot

@numba.njit
def distances_numba_array(cluster):
    # Original: diff = cluster[:, np.newaxis, :] - cluster[np.newaxis, :, :]
    # Since np.newaxis is not supported, use reshape to do this
    diff = (cluster.reshape(cluster.shape[0], 1, cluster.shape[1]) -
            cluster.reshape(1, cluster.shape[0], cluster.shape[1]))
    mat = (diff * diff)
    # Original: mat = mat.sum(-1)
    # Since axis argument is not supported, write the loop out
    out = np.empty(mat.shape[:2], dtype=mat.dtype)
    for i in np.ndindex(out.shape):
        out[i] = mat[i].sum()

    return np.sqrt(out)

@numba.njit
def potential_numba_array(cluster):
    d = distances_numba_array(cluster)
    # Original: dtri = np.triu(d)
    # np.triu is not supported; so write loop to clear the
    # lower triangle
    for i in range(d.shape[0]):
        for j in range(d.shape[1]):
            if i > j:
                d[i, j] = 0
    # Original: lj_numba_array(d[d > 1e-6]).sum()
    # d[d > 1e-6] is not supported due to the indexing with boolean
    # array.  Replace with custom loop.
    energy = 0.0
    for v in d.flat:
        if v > 1e-6:
            energy += lj_numba_array(v)
    return energy

In [ ]:
%timeit potential_numba_array(cluster)

### Task: Create a function that compares the speed of NumPy and Numba implementations of the Lennard-Jones potential

Use random square matrices of size 500, 1000, and 1500. Time the intialization and multiplication collectively using `time.time()`. Hints: [`rand()`](https://docs.scipy.org/doc/numpy-1.14.0/reference/generated/numpy.random.rand.html) and [`matmul()`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.matmul.html).